# Azure AI агенти с поддръжка на Model Context Protocol (MCP) - Python

Този notebook демонстрира как да използвате Azure AI агенти с инструменти на Model Context Protocol (MCP) в Python. Показва как да създадете интелигентен агент, който може да използва външни MCP сървъри (като Microsoft Learn) за разширени възможности с помощта на удостоверяване без ключове.


## Инсталиране на необходимите Python пакети

Първо, трябва да инсталираме необходимите Python пакети:
- **azure-ai-projects**: Основен SDK за Azure AI Projects
- **azure-ai-agents**: SDK за Azure AI Agents за създаване и управление на агенти
- **azure-identity**: Осигурява безключова автентикация с DefaultAzureCredential
- **mcp**: Имплементация на Model Context Protocol за Python


## Предимства на удостоверяването без ключове

Тази тетрадка демонстрира **удостоверяване без ключове**, което предоставя няколко предимства:
- ✅ **Няма нужда от управление на API ключове** - Използва удостоверяване, базирано на Azure идентичност
- ✅ **Подобрена сигурност** - Няма тайни, съхранявани в кода или конфигурационни файлове
- ✅ **Автоматично обновяване на идентификационни данни** - Azure се грижи за управлението на жизнения цикъл на идентификациите
- ✅ **Контрол на достъпа, базиран на роли** - Използва Azure RBAC за детайлно управление на разрешенията
- ✅ **Поддръжка на множество среди** - Работи безпроблемно както в разработка, така и в продукция

`DefaultAzureCredential` автоматично избира най-добрия наличен източник на идентификационни данни:
1. **Управлявана идентичност** (когато се изпълнява в Azure)
2. Удостоверения от **Azure CLI** (по време на локална разработка)
3. Удостоверения от **Visual Studio**
4. **Променливи на средата** (ако са конфигурирани)
5. Удостоверяване чрез **интерактивен браузър** (като резервен вариант)


## Настройка на удостоверяване без ключ

**Предварителни изисквания за удостоверяване без ключ:**

### За локална разработка:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### За среди в Azure:
- Активирайте **Системно зададена управлявана идентичност** на вашия Azure ресурс
- Присвоете подходящи **RBAC роли** на управляваната идентичност:
  - `Cognitive Services OpenAI User` за достъп до Azure OpenAI
  - `AI Developer` за достъп до проекти в Azure AI

### Променливи на средата (по избор):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Не са необходими API ключове или низове за връзка!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Импортиране на необходимите библиотеки

Импортирайте необходимите Python модули:
- **os, time**: Стандартни Python библиотеки за променливи на средата и забавяния
- **AIProjectClient**: Основен клиент за Azure AI Projects
- **DefaultAzureCredential**: Безключова автентикация за Azure услуги
- **Класове, свързани с MCP**: За създаване и управление на MCP инструменти и обработка на одобрения


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Конфигуриране на настройките на MCP сървъра

Настройте конфигурацията на MCP сървъра, като използвате променливи на средата с резервни стойности по подразбиране:
- **MCP_SERVER_URL**: URL адресът на MCP сървъра (по подразбиране Microsoft Learn API)
- **MCP_SERVER_LABEL**: Етикет за идентифициране на MCP сървъра (по подразбиране "mslearn")

Този подход позволява гъвкава конфигурация в различни среди.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Създаване на клиент за проект в Azure AI (Автентикация без ключ)

Инициализирайте клиента за проект в Azure AI, използвайки **автентикация без ключ**:
- **endpoint**: URL адрес на крайна точка на проекта в Azure AI Foundry
- **credential**: Използва `DefaultAzureCredential()` за сигурна автентикация без ключ
- **Не са необходими API ключове**: Автоматично открива и използва най-добрите налични идентификационни данни

**Процес на автентикация:**
1. Проверява за управлявана идентичност (в среди на Azure)
2. Преминава към идентификационни данни от Azure CLI (за локална разработка)
3. Използва други налични източници на идентификационни данни при необходимост

Този подход елиминира нуждата от управление на API ключове или низове за връзка във вашия код.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Създаване на дефиниция за MCP инструмент

Създайте MCP инструмент, който се свързва с MCP сървъра на Microsoft Learn:
- **server_label**: Идентификатор за MCP сървъра
- **server_url**: URL адрес на MCP сървъра
- **allowed_tools**: Незадължителен списък за ограничаване на инструментите, които могат да се използват (празен списък позволява използването на всички инструменти)

Този инструмент ще позволи на агента да получава достъп до документацията и ресурсите на Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Създаване на агент и провеждане на разговор (Работен процес без ключове)

Този подробен раздел демонстрира пълния **работен процес на агент без ключове**:

1. **Създаване на AI агент**: Настройте агент с модела GPT-4.1 nano и инструменти MCP
2. **Създаване на нишка**: Създайте нишка за разговор за комуникация
3. **Изпращане на съобщение**: Попитайте агента за разликите между Azure OpenAI и OpenAI
4. **Одобряване на инструменти**: Автоматично одобрявайте използването на инструменти MCP, когато е необходимо
5. **Проследяване на изпълнението**: Наблюдавайте напредъка на агента и предприемайте необходимите действия
6. **Показване на резултати**: Представете подробности за разговора и използването на инструменти

**Характеристики без ключове:**
- ✅ **Без твърдо кодирани тайни** - Цялата автентикация се обработва чрез Azure идентичност
- ✅ **Сигурност по подразбиране** - Използва контрол на достъпа, базиран на роли
- ✅ **Опростено внедряване** - Не е необходимо управление на идентификационни данни
- ✅ **Подходящо за одит** - Целият достъп се проследява чрез Azure идентичност

Агентът ще използва инструменти MCP за достъп до ресурси на Microsoft Learn с пълна сигурност и без управление на API ключове.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
